In [1]:
from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef

In [2]:
# load the keyword instances graph and bnode graph because this is the data we want to link (decided to focus ont these two for now and then later attempt with mesh)
kw = Graph()
ne = Graph()
kw.parse("./ParsingSpaqrlIntoTTLFile/Output-Graphs/HypothesisKeywordInstances.ttl", format="turtle")
ne.parse("./ParsingSpaqrlIntoTTLFile/Output-Graphs/bnode_graph_new.ttl", format="turtle")

<Graph identifier=N25ca0f69383140c3936cc7145735a630 (<class 'rdflib.graph.Graph'>)>

In [3]:
# get the namespaces for running the sparql queries in python
def create_namespace(graph, namespace, prefix):

    ns = Namespace(namespace)
    graph.namespace_manager.bind(prefix, namespace)
    
    return ns


hyp_namespace = create_namespace(ne, "http://example.org/hypothesis_ontology/", 'hyp')
oa_namespace = create_namespace(ne, "http://www.w3.org/ns/oa#", 'oa')

hyp_namespace = create_namespace(kw, "http://example.org/hypothesis_ontology/", 'hyp')
oa_namespace = create_namespace(kw, "http://www.w3.org/ns/oa#", 'oa')
# covidpr_namespace = create_namespace(bnode_g, "http://ns.inria.fr/covid19/property/", 'covidpr')
# dct_namespace = create_namespace(bnode_g, "http://purl.org/dc/terms/", 'dct')
# covid_namespace = create_namespace(bnode_g, "http://ns.inria.fr/covid19/", 'covid')
# prov_namespace = create_namespace(bnode_g, 'http://www.w3.org/TR/prov-o/', 'prov')

Notes: From this point, I was trying to get the data from the graph and retrieve it in a way that makes it easier to compare the literals of the instances. First I though going through the triples is a possible way, but now I think performing sparql queries and organzing the retrieved data might be more promising... I decided to try storing the info in dictionaries for now and then my idea was to loop through them both and perform some comparing methods. Through this, I would store the  keyword uri and the named entity uri that shared similar literals in a tuple. Finally, I would create triples of these with rdflib through g.add((keywordURI, skos.related, namedentityURI)) and save these to a turtle file. 

In [29]:
def get_abstracts(graph):
    abstracts = []
    for s, p, o in graph.triples( (None, oa.hasSource, None) ):
        keyword = graph.value(s,p)
        abstracts.append(keyword)
    return abstracts

# get_abstracts(kw)

In [24]:
len(set(get_abstracts(ne)))

1578

In [7]:
ne_q = ne.query(
"""
prefix dct: <http://purl.org/dc/terms/> 
prefix hyp: <http://example.org/hypothesis_ontology/> 
prefix oa: <http://www.w3.org/ns/oa#>
SELECT ?ne ?literal ?abstract
WHERE {
  ?ne oa:hasSource ?abstract; oa:exact ?literal.
}
"""
)

ne_data = []
for i in ne_q:
    ne_dict = dict()
    ne_dict['ne_id'] = i.ne
    ne_dict['literal'] = i.literal
    ne_dict['abstract'] = i.abstract
    ne_data.append(ne_dict)

In [51]:
kw_q = kw.query(
"""
prefix dct: <http://purl.org/dc/terms/> 
prefix hyp: <http://example.org/hypothesis_ontology/> 
prefix oa: <http://www.w3.org/ns/oa#>
SELECT ?kw ?literal ?abstract
WHERE {
  ?hyp oa:hasSource ?abstract; hyp:contains ?kw.
  ?kw oa:hasTarget ?literal.
}
"""
)

kw_data = []
for i in kw_q:
    kw_dict = dict()
    kw_dict['kw_id'] = i.ne
    kw_dict['literal'] = i.literal
    kw_dict['abstract'] = i.abstract
    kw_data.append(kw_dict)

http://example.org/hypothesis_ontology/48b64108-d9d9-4add-9e53-e0ff30530766#keyword http://ns.inria.fr/covid19/85600d58166573ed016072d757f3b6e133098b74#abstract
http://example.org/hypothesis_ontology/cb77bc05-9eca-4c0f-8467-1c3a716fbcdf#keyword http://ns.inria.fr/covid19/bca9f4ffe445882f2de559b938d83a69e90d2224#abstract
http://example.org/hypothesis_ontology/3741c880-8cf8-43c5-96fd-6ff0ebfa3b2e#keyword http://ns.inria.fr/covid19/744b36e6454853e3da80621600705f6ba215e6c5#abstract
http://example.org/hypothesis_ontology/f0e8d3aa-fe7c-453c-b21f-d3d837245f8f#keyword http://ns.inria.fr/covid19/390b508dfbde0fbf9b91e67cc77fcd3bf0e391ed#abstract
http://example.org/hypothesis_ontology/0900bff6-47b7-4038-8fb8-e80cc5e92fab#keyword http://ns.inria.fr/covid19/2757708c4f72260abefb9d74302d868165cbc1a4#abstract
http://example.org/hypothesis_ontology/e1e42625-2131-4eb6-a4c5-4b42910355bb#keyword http://ns.inria.fr/covid19/aa5c3ba1a343c13446ce37073c2e03cf91032f28#abstract
http://example.org/hypothesis_onto

In [12]:
ne_data[10]['ne_id']

rdflib.term.URIRef('http://example.org/hypothesis_ontology/29de8576-e665-4418-be64-f754b3f9b1c0#namedEntity')